In [1]:
!pip install -q transformers torch pandas numpy scikit-learn tqdm matplotlib huggingface_hub peft

import os, subprocess, time

def stage_to_local(src_dir: str, dst_dir: str, marker_name=".staged_ok"):
    os.makedirs(dst_dir, exist_ok=True)
    marker = os.path.join(dst_dir, marker_name)

    # 이미 스테이징 완료면 스킵
    if os.path.exists(marker):
        print(f"[stage] already staged: {dst_dir}")
        return

    print(f"[stage] rsync from\n  {src_dir}\n  -> {dst_dir}")
    t0 = time.time()

    # rsync는 증분 복사라서 중간에 끊겨도 다시 실행하면 이어서 빨리 맞춰짐
    cmd = f"rsync -a --info=progress2 '{src_dir.rstrip('/')}/' '{dst_dir.rstrip('/')}/'"
    subprocess.run(["bash", "-lc", cmd], check=True)

    # 마커 생성
    with open(marker, "w") as f:
        f.write("ok\n")

    print(f"[stage] done in {time.time()-t0:.1f}s")

# ====== 0) 구글드라이브 마운트 & code 경로 설정 ======
import os, sys

# Colab이 아니면 drive mount가 실패할 수 있음(로컬/엘리스/기타 환경 호환)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('[warn] drive mount skipped:', e)

# 프로젝트 루트(Drive 기준). 필요하면 여기만 바꾸면 됨.
DRIVE_ROOT = '/content/drive/MyDrive/TRAITHON'
DRIVE_CODE = os.path.join(DRIVE_ROOT, 'code')
DRIVE_ADAPTER = os.path.join(DRIVE_ROOT, 'models/adapter')

# /content 로컬 스테이징(빠름)
LOCAL_CODE       = '/content/code'
LOCAL_BASE_MODEL = '/content/A.X-4.0-Light'
LOCAL_ADAPTER    = '/content/ax4_adapter_final'

# (선택) Drive에 있는 모듈/어댑터를 /content로 스테이징 (rsync 기반, 증분 복사)
if os.path.exists(DRIVE_CODE):
    try:
        stage_to_local(DRIVE_CODE, LOCAL_CODE)
    except Exception as e:
        print('[warn] code staging failed:', e)

if os.path.exists(DRIVE_ADAPTER):
    try:
        stage_to_local(DRIVE_ADAPTER, LOCAL_ADAPTER)
    except Exception as e:
        print('[warn] adapter staging failed:', e)

# 실제로 import에 쓸 CODE_DIR 결정 (우선순위: /content/code -> Drive/code -> ./code)
if os.path.exists(LOCAL_CODE):
    CODE_DIR = LOCAL_CODE
elif os.path.exists(DRIVE_CODE):
    CODE_DIR = DRIVE_CODE
else:
    CODE_DIR = './code'

if CODE_DIR not in sys.path:
    sys.path.insert(0, CODE_DIR)

print('CODE_DIR =', CODE_DIR)
print('LOCAL_ADAPTER =', LOCAL_ADAPTER)

# 데이터셋 경로 추가 설정
DRIVE_DATA = os.path.join(DRIVE_ROOT, 'data/golden_set_v1_TL')
LOCAL_DATA = '/content/data'

if os.path.exists(DRIVE_DATA):
    try:
        # 데이터셋 5,000개를 /content/data로 복사 (이게 가장 중요!)
        stage_to_local(DRIVE_DATA, LOCAL_DATA)
    except Exception as e:
        print('[warn] data staging failed:', e)

# 이후 6번째 셀에서 DATA_PATH를 LOCAL_DATA로 바꿔서 사용하세요.
DATA_PATH = LOCAL_DATA

Mounted at /content/drive
[stage] rsync from
  /content/drive/MyDrive/TRAITHON/code
  -> /content/code
[stage] done in 10.1s
[stage] rsync from
  /content/drive/MyDrive/TRAITHON/models/adapter
  -> /content/ax4_adapter_final
[stage] done in 8.4s
CODE_DIR = /content/code
LOCAL_ADAPTER = /content/ax4_adapter_final
[stage] rsync from
  /content/drive/MyDrive/TRAITHON/data/golden_set_v1_TL
  -> /content/data
[stage] done in 168.7s


In [2]:
import unicodedata
import ax4_clickbait_scorer as sc
import clickbait_preprocess as cp
from transformers import AutoTokenizer

# 1. 기존 실험과 동일한 토크나이저 로드 (sc._TOKENIZER 할당)
# 모델 경로나 허깅페이스 ID를 넣어주세요.
MODEL_PATH = "/content/A.X-4.0-Light"
try:
    sc._TOKENIZER = AutoTokenizer.from_pretrained(MODEL_PATH)
    print("✅ sc._TOKENIZER 로드 성공")
except:
    sc._TOKENIZER = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
    print("⚠️ 기본 토크나이저로 대체 로드됨")

# 2. 운영용 Sanitizer 함수 (F1, F4 통합 대응)
def run_operational_sanitizer(text):
    """
    모든 입력 텍스트에 대해 제로폭 문자(F1) 및 결합 문장부호(F4)를 제거하여
    잠재적 난독화를 해제한 텍스트를 반환함.
    """
    if not text: return ""
    # F1(Cf: 제어 문자) 및 F4(Mn, Mc, Me: 결합 마크) 카테고리 제거
    chars = [c for c in text if unicodedata.category(c) not in ['Cf', 'Mn', 'Mc', 'Me']]
    return "".join(chars)

print("✅ 운영 감시 시스템 준비 완료")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

⚠️ 기본 토크나이저로 대체 로드됨
✅ 운영 감시 시스템 준비 완료


In [3]:
import json
import pandas as pd
from datetime import datetime, timezone
from tqdm.auto import tqdm

json_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.json')]
operational_logs = []

print(f"🚀 실시간 운영 데이터 모니터링 시작 (총 {len(json_files)}건)")

# 규칙: gain_tok >= 5 이면 '난독화 공격 의심'으로 탐지 [cite: 265]
THRESHOLD = 5

for filename in tqdm(json_files):
    with open(os.path.join(DATA_PATH, filename), 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    # 1. 원본 입력 데이터 확보 (제목 + 본문) [cite: 97]
    src = raw_data.get("sourceDataInfo", {})
    raw_input = (src.get("newsTitle", "") + " " + src.get("newsContent", "")).strip()
    if not raw_input: continue

    # 2. Sanitizer 적용 (정제 과정) [cite: 94]
    sanitized_input = run_operational_sanitizer(raw_input)

    # 3. 토큰 차이 계산 (sc._TOKENIZER 방식) [cite: 5, 232]
    tokens_raw = len(sc._TOKENIZER.encode(raw_input))
    tokens_clean = len(sc._TOKENIZER.encode(sanitized_input))
    gain_tok = tokens_raw - tokens_clean

    # 4. 탐지 및 상태 결정
    status = "ALERT" if gain_tok >= THRESHOLD else "PASS"

    operational_logs.append({
        'timestamp': datetime.now(timezone.utc).isoformat(),
        'article_id': filename,
        'gain_tok': gain_tok,
        'status': status,
        'action': 'BLOCKED' if status == "ALERT" else 'NONE' # 임계값 초과 시 차단 조치 [cite: 265]
    })

# 데이터프레임 변환
df_ops_log = pd.DataFrame(operational_logs)
print(f"\n✅ 모니터링 완료. 탐지 건수: {len(df_ops_log[df_ops_log['status'] == 'ALERT'])}건")

🚀 실시간 운영 데이터 모니터링 시작 (총 5000건)


  0%|          | 0/5000 [00:00<?, ?it/s]


✅ 모니터링 완료. 탐지 건수: 0건


In [4]:
# 1. 저장 경로 생성
os.makedirs('evidence/output6/O_observed/table1', exist_ok=True)
os.makedirs('evidence/output6/O_observed/table4', exist_ok=True)

# 2. 운영 모니터링 데이터 (Table 1: 운영 성능 증거) [cite: 3]
df_ops_log.to_csv('evidence/output6/O_observed/table1/weekly_eval_ax4_obf.csv', index=False)

# 3. 연속 감사 추적 로그 (Table 4: 감사 기록 증거) [cite: 10, 265]
# 감사 기록 ID 형식 예: AUD-2026-01-16-001
audit_log_path = 'evidence/output6/O_observed/table4/trigger_result_obf.log'
with open(audit_log_path, 'w') as f:
    f.write("=== [TRAITHON] Adversarial Obfuscation Operational Audit Log ===\n")
    f.write(f"Monitor Window: {datetime.now(timezone.utc).date()} (Rolling 7d)\n")
    f.write(f"Detection Rule: Gain_Tok >= {THRESHOLD}\n")
    f.write(f"Total Scanned: {len(df_ops_log)}\n")
    f.write(f"Alert Count: {len(df_ops_log[df_ops_log['status'] == 'ALERT'])}\n")
    f.write(f"False Positive Rate (on Golden Set): {len(df_ops_log[df_ops_log['status'] == 'ALERT']) / len(df_ops_log):.4f}\n")
    f.write("-" * 50 + "\n")

    # ALERT가 발생한 상세 리스트 상위 10건 기록
    alerts = df_ops_log[df_ops_log['status'] == 'ALERT'].head(10)
    if not alerts.empty:
        f.write("Detailed Incident Preview (Top 10):\n")
        f.write(alerts.to_string(index=False))
    else:
        f.write("No incidents detected in this window.\n")

print(f"✅ 산출물6용 운영 로그 생성 완료:")
print(f"- 운영 지표 데이터: evidence/output6/O_observed/table1/weekly_eval_ax4_obf.csv")
print(f"- 감사 추적 로그: {audit_log_path}")

✅ 산출물6용 운영 로그 생성 완료:
- 운영 지표 데이터: evidence/output6/O_observed/table1/weekly_eval_ax4_obf.csv
- 감사 추적 로그: evidence/output6/O_observed/table4/trigger_result_obf.log


In [6]:
import json, os, random, unicodedata
import pandas as pd
from datetime import datetime, timezone

# 1. 인시던트 발생용 공격 함수 (F1, F4)
def simulate_attack(text):
    # 제로폭 문자(F1) 10개 강제 주입하여 Gain_Tok >= 5 유도
    f1_chars = ['\u200B', '\u200C', '\u200D']
    chars = list(text)
    for _ in range(10):
        chars.insert(random.randint(0, len(chars)), random.choice(f1_chars))
    return "".join(chars)

# 2. 시뮬레이션 실행 (기존 운영로그 생성 코드 뒤에 추가)
incident_samples = []
# 정상 데이터 중 5개를 뽑아 공격 시뮬레이션 수행
target_files = json_files[:5]

for filename in target_files:
    with open(os.path.join(DATA_PATH, filename), 'r') as f:
        data = json.load(f)
    raw_input = (data['sourceDataInfo']['newsTitle'] + " " + data['sourceDataInfo']['newsContent']).strip()

    # 공격 주입
    attacked_text = simulate_attack(raw_input)
    sanitized_text = run_operational_sanitizer(attacked_text)

    gain_tok = len(sc._TOKENIZER.encode(attacked_text)) - len(sc._TOKENIZER.encode(sanitized_text))

    incident_samples.append({
        'timestamp': datetime.now(timezone.utc).isoformat(),
        'article_id': f"ATTACK_{filename}",
        'gain_tok': gain_tok,
        'status': 'ALERT',
        'action': 'BLOCKED'
    })

# 시뮬레이션 로그 저장
df_incident = pd.DataFrame(incident_samples)
df_incident.to_csv('evidence/output6/O_observed/table3/incident_simulation_log.csv', index=False)
print("⚠️ 인시던트 시뮬레이션 완료: ALERT 로그 5건 생성됨")

⚠️ 인시던트 시뮬레이션 완료: ALERT 로그 5건 생성됨
